## Expired Futures
When searching for an expired asset, you will need to specify the date range based on the ExpiryDate property.

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.Data.Content, 1.0.0-beta4"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Microsoft.Data.Analysis, 0.20.1 Refinitiv.Data.Content, 1.0.0-beta4

Loading extensions from `C:\Users\U8007876\.nuget\packages\microsoft.data.analysis\0.20.1\interactive-extensions\dotnet\Microsoft.Data.Analysis.Interactive.dll`

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.Data.Content.SearchService;
using Refinitiv.Data.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Data;
using System;
using Microsoft.DotNet.Interactive.Formatting;
using static Microsoft.DotNet.Interactive.Formatting.PocketViewTags;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<DataTable>((dt, writer) =>
{
    if (dt.Columns.Count > 0)
    {
        var rows = new List<ICollection<IHtmlContent>>();

        // Process Each data row
        foreach(var row in dt.Rows.Cast<DataRow>())
        {
            var cells = new List<IHtmlContent>();
            foreach (var cell in row.ItemArray)
            {
                switch (cell)
                {
                    case null:
                        break;
                    case JValue val:
                        cells.Add(td(val.ToString()));
                        break;
                    case IEnumerable<JToken> list:
                        cells.Add(td($"[{string.Join(", ", list.Select(node => $"{node}"))}]"));
                        break;
                    default:
                        cells.Add(td(cell));
                        break;
                }                
            }
            rows.Add(cells);
        }
        
        // Header
        var headers = new List<IHtmlContent>();
        foreach (DataColumn col in dt.Columns)
            headers.Add(th(col.ColumnName) as IHtmlContent);

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
DesktopSession.Definition().GetSession().OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                                        .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})"))
                                        .Open();

2023-02-21 3:58:42 PM:DesktopSession. (State: DesktopSession is Pending)
2023-02-21 3:58:44 PM:DesktopSession. (Event: {
  "Contents": "Desktop Session Successfully Authenticated"
})
2023-02-21 3:58:44 PM:DesktopSession. (State: DesktopSession is Opened)


#### Example - Expired Vodafone futures in the month of Nov 2019.

In [5]:
var filter = "TickerSymbol eq 'vod' and ExpiryDate lt 2019-12-01 and ExpiryDate ge 2019-11-01";
var select = "DTSubjectName, ExchangeName, RIC, RCSAssetCategoryLeaf, AssetState, ExpiryDate, UnderlyingQuoteRIC";

var response = Search.Definition(Search.View.DerivativeQuotes).Filter(filter)
                                                              .Select(select)
                                                              .GetData();
response.Data.Table

Johannesburg Stock Exchange Vodacom Group SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODQX9^1,Equity Future,DC,2019-11-21 12:00:00 AM,[VODJ.J]
Johannesburg Stock Exchange Vodacom Group Limited Cash Settled SSF Equity Future Nov 2019,Johannesburg Stock Exchange,VODSX9^1,Equity Future,DC,2019-11-21 12:00:00 AM,[VODJ.J]


#### Example - Expired Crude Oil Futures in the last year

In [6]:
// Build expiry date range expression
var dateOneYearAgo = DateTime.Now.AddYears(-1);
var dateRange = $"ExpiryDate ge {dateOneYearAgo:yyyy-MM-dd} and ExpiryDate lt {DateTime.Now:yyyy-MM-dd}";

In [7]:
filter = $@"RCSUnderlyingProductName xeq 'Energy' and
            SearchAllCategoryv3 eq 'Commodities' and
            RCSAssetCategoryName eq 'Derivative\Future' and RCSUnderlyingProductName eq 'Crude Oil' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";

response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Table

NYMEX WTI Light Sweet Crude Oil Calendar Swap US Quarterly Mar 2023,CLCALQH3,2022-12-31 12:00:00 AM,Crude Oil
ICE Brent Crude Electronic Energy Fair Value Future Nov 2022,LCOFVX2,2022-09-30 12:00:00 AM,Brent Crude
Shanghai International Energy Exchange Medium Sour Crude Oil Fair Value Future Oct 2022,ISCFVV2,2022-09-23 12:00:00 AM,Crude Oil
NYMEX WTI Light Sweet Crude Oil Calendar Swap US Monthly Aug 2022,CLCALMQ2,2022-07-31 12:00:00 AM,Crude Oil
ICE WTI Crude Electronic Energy Fair Value Future May 2022,WTCLFVK2,2022-04-19 12:00:00 AM,US Light Sweet Crude


#### Example - Expired Electricity Futures in the last year

In [8]:
filter = $@"SearchAllCategoryv3 eq 'Commodities' and
            RCSUnderlyingProductName eq 'Power' and
            ExchangeName xeq 'Source is a Refinitiv Contributor' and {dateRange}";

response = Search.Definition(Search.View.SearchAll).Filter(filter)
                                                   .OrderBy("ExpiryDate desc")
                                                   .Top(30)
                                                   .Select("DTSubjectName, RIC, ExpiryDate, RCSUnderlyingProductLeaf")
                                                   .GetData();
response.Data.Table

Power Nordic Tallinn Fair Value Continuation 45,TRTALBFVDc45,2023-02-20 12:00:00 AM,Power
TR TAL FV W7,TRTALBFVWc7,2023-02-19 12:00:00 AM,Power
Power Nordic Tallinn Fair Value Continuation 44,TRTALBFVDc44,2023-02-19 12:00:00 AM,Power
Power Nordic Tallinn Fair Value Continuation 43,TRTALBFVDc43,2023-02-18 12:00:00 AM,Power
Power Nordic Tallinn Fair Value Weekly Continuation 1,TRTALBFVWc1,2023-01-08 12:00:00 AM,Power
Power Nordic Tallinn Fair Value Weekly Continuation 0,TRTALBFVWc0,2023-01-08 12:00:00 AM,Power
ICE-US FCOJ-A Futures Electronic Commodity Fair Value Future Nov 2022,OJFVX2,2022-11-09 12:00:00 AM,Power
